# Food For Thought 
## Digging into the dataset

The first step of this project aims at understanding the dataset we have chosen ([Open Food Facts Database](https://world.openfoodfacts.org/)), to check whether it is suitable for the kind of analysis we want to develop.

As a reminder, we would like to focus our research on two main topics:
1. Impact of food on environment
2. Impact of food on the health

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
from mpl_toolkits.mplot3d import Axes3D
from difflib import get_close_matches

import pickle

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import findspark
findspark.init()

from pyspark.sql import *

import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
%matplotlib inline


In [2]:
# functions to save and load intermediate results in pickle format
def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

def save_pickle(result, file_path = 'pickle'):
    with open(file_path, 'wb') as file:
        pickle.dump(result, file)

In [4]:
style.use('seaborn-notebook') #sets the size of the charts
style.use('ggplot')

In [5]:
DATA_FOLDER = './data/'
data = spark.read.csv(DATA_FOLDER +'en.openfoodfacts.org.products.csv',header=True,sep='\t')          

### Brand vs Nutrition vs Food Category in France 

In [6]:
brand = data.filter("countries_en=='France'")\
                    .select('brands', 'nutrition_grade_fr', 'pnns_groups_1')\
                    .dropna(how='all')

In [7]:
brand = brand.withColumn('pnns_groups_1',(F.regexp_replace('pnns_groups_1', '-', ' ')))
square_udf_int = F.udf(lambda z: z.title())
brand = brand.withColumn('pnns_groups_1',square_udf_int('pnns_groups_1'))

In [8]:
# we convert the spark dataframe to a pandas dataframe
brand_df = brand.toPandas()

# we store it in the current folder 
brand_df.to_pickle('brand_milestone3_df')

Py4JJavaError: An error occurred while calling o59.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 1.0 failed 1 times, most recent failure: Lost task 3.0 in stage 1.0 (TID 4, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 238, in main
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 690, in read_int
    length = stream.read(4)
  File "C:\Users\Tom\Anaconda3\lib\socket.py", line 586, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:330)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(Pyt

In [ ]:
# we can now read it everytime we want to continue the analysis
brand_df = pd.read_pickle('brand_milestone3_df')

The problem is that the 'brands' tags are not always consistent. For example we can find the brand tag 'carrefour', "Carrefour", "carrfour,bio carrefour' ect. 
Hence we first clean the brands using a list of food brands present in France and a function to match the 'brands' tag with the closest brand in the list. 

In [ ]:
# list of food brands present in France ('Coca Cola', 'Fleury Michon', 'Carrefour' ect.)
list_brand_file = open("brand_list.txt", "r")
list_brand = list_brand_file.read().split('\n')

# Function which find for each value (string) in the column 'brands' the closest brand (string) in 'list_brand'.
# Note: the function 'get_close_matches' has been imported from the open-source 'difflib' library.
def clean_brand(brand_name):
    
    # brand name can be of the form 'Carrefour' (one word) or 'Carrefour, Bio Carrefour' (multiple words)
    # hence we split the string w.r.t the symbol ',' and find the best matching brand word for each split.
    best=[]
    for word in brand_name.split(','):
        
        # the cutoff control for the False Positives possibilities in the 'list_brand' 
        # that don’t score at least the cutoff are ignored.
        # n controls for the the number of possibilities (whose score is higher than the cutoff value).
        best.append(get_close_matches(word, list_brand, n=1, cutoff=0.7))
    
    # remove empty sublists and unravel
    best = [brand for sublist in best for brand in sublist if sublist]
    
    # if there was 
    if best:
        return best[0]
    else:
        return None


In [ ]:
# we create a new column with the new "consistent" brand tag. 
# takes some minutes to run...
brand_df['new_brands'] = brand_df['brands'].apply(lambda x: clean_brand(x))

In [ ]:
# we can drop the old column 'brands' since we have the new ones.
brand_df_cleaned = brand_df.dropna().drop('brands',1).rename(columns={'new_brands':'brands'})
brand_df_cleaned.head()

In [ ]:
# here we count the number of products for each brand. We consider that brand have enough products to be analyzed from 100 products
brand_count = brand_df_cleaned.groupby('brands').count().sort_values('nutrition_grade_fr', ascending=False)
brand_count['enough_products']= brand_count.apply(lambda x: x>100)
brand_count = brand_count[brand_count.enough_products]
brand_count.head()

The most abundant brands are Carrefour, Auchan and U. There are the brands of the biggest distributors (supermarkets) in France. 

In [ ]:
# here we add the column "enough products" to the dataframe with the cleaned brand.
brand = brand_df_cleaned.join(brand_count[['enough_products']], on='brands')
brand = brand.dropna().drop('enough_products',1)
brand.head()

In [ ]:
# here we create dummy variables from the 'nutrition_grade_fr' whose value can be:
# - 'a' (very good product for health) 
# - 'b' (good product)
# - 'c' ('neutral product')
# - 'd' ('not so good product')
# - 'e' (bad product for health)

brand_expanded = pd.get_dummies(brand.set_index('brands')).reset_index()
brand_expanded = brand_expanded.rename(columns={'nutrition_grade_fr_a':'a',
                               'nutrition_grade_fr_b':'b',
                               'nutrition_grade_fr_c':'c',
                               'nutrition_grade_fr_d':'d',
                               'nutrition_grade_fr_e':'e'})
brand_expanded.head(5)

### Food quality in the whole dataset

In [ ]:
count = brand_df[['nutrition_grade_fr','brands']].groupby('nutrition_grade_fr').count().rename(columns={'brands':'count'})
count.plot(kind='bar')
plt.title('Distribution of the nutrition grades among the whole dataset')
plt.xlabel('nutrition grade') 
plt.show()

### Food quality for each brand

In [ ]:
# extract the total number of products for each brands
brand_count = brand.groupby('brands').count().rename(columns={'nutrition_grade_fr':'total'})
brand_count.head()

In [ ]:
# here we count the nutrition grade occurences for each brand
brand_expanded_count = brand_expanded.groupby('brands').sum()
brand_expanded_count.head(3)

In [ ]:
# we add the column total (with the total number of products sold for each brand) to further extract the ratio of each grade instead of the raw count.
brand_expanded_count = brand_expanded_count.join(brand_count)#.rename(columns = {'nutrition_grade_fr':'total'})
brand_expanded_count.head()

In [ ]:
# we convert the counts into ratios. (ex: ratio for 'a' = count for 'a' / total number of products)
ratio = brand_expanded_count.copy()
ratio[['a','b','c','d','e']] = brand_expanded_count[['a','b','c','d','e']].div(brand_expanded_count['total'].values,axis=0)
ratio = ratio.sort_values('total',ascending=False)

In [ ]:
ratio.reset_index().loc[:10].drop('total',1).plot(kind='bar', x='brands',figsize=[15,10], color=sns.color_palette("RdBu_r", 5))
plt.ylabel('proportion (in %)')
plt.title('Health profile of the 10 biggest food brands in France.')
plt.show()

We tought we could reduce the number of possible scores to allow even simplier analysis. From the set of 5 grades (a,b,c,d,e) we go to a set of three grades:
- 'healthy' food if a or b grades
- 'neutral' food if c grade
- 'junk' food if d or e grades

In [ ]:
# function to transform the scores contained in a row dataframe. 
def get_new_score(df_row):
    if df_row['a']==1 or df_row['b']==1:
        new_score = 'healthy' 
    if df_row['c']==1:
        new_score = 'neutral' 
    if df_row['d']==1 or df_row['e']==1:
        new_score = 'junk'
    return new_score

In [ ]:
# here we pass for the "old" scores to the "new" ones
brand_with_new_score = brand_expanded.copy()

# we create a new column to store the "new" scores
brand_with_new_score['new_score'] = brand_expanded.apply(get_new_score, axis=1)

# we convert those "new" scores to dummy variables, as before for the "old" ones
brand_with_new_score = pd.get_dummies(brand_with_new_score[['brands','new_score']].set_index('brands')).reset_index()

# we rename the columns
brand_with_new_score = brand_with_new_score.rename(columns={'new_score_healthy':'healthy',
                                     'new_score_junk':'junk',
                                     'new_score_neutral':'neutral'})[['brands','healthy','neutral','junk']]
brand_with_new_score.head()

In [ ]:
# we can group the products by brands as before to get 
brand_with_new_score = brand_with_new_score.groupby('brands').sum()
brand_with_new_score.head(3)

In [ ]:
# we get the ratios instead of the raw counts, as before
brand_with_new_score = brand_with_new_score.join(brand_count)
ratio = brand_with_new_score.copy()
ratio[['healthy','neutral','junk']] = brand_with_new_score[['healthy','neutral','junk']].div(brand_with_new_score['total'].values,axis=0)
ratio = ratio.sort_values('total',ascending=False)

In [ ]:
# final plot
ratio.reset_index().loc[:20].drop('total',1).plot(kind='bar', x='brands',figsize=[15,10], color=sns.color_palette("RdBu_r", 3))
plt.ylabel('proportion (in %)')
plt.show()

We observe for example that for almost all the biggest food brands present in France the prominent nutrition category is "junk" food. We don't eat that well in France... 

The two biggest exceptions are 'Danone' and 'Picard'. They mainly sell products from milk and frozen food which could be excpected to be healthier than the products sold by Fleury Michon (mainly transformed products from meat).  

Interestingly the 'Bio Village' brand solds a lot of 'junk' food... Even more than 'neutral' food.

### Analysis of the case 'Bio Village'

In [ ]:
bioVillage = data.filter("countries_en=='France'").select('brands', 'nutrition_grade_fr', 'pnns_groups_1')
bioVillage_df = bioVillage.toPandas()

In [ ]:
bioVillage_df = bioVillage_df.dropna(how='all')

In [ ]:
#bioVillage_df.to_pickle('bioVillage_df')
bioVillage_df = pd.read_pickle('bioVillage_df')
bioVillage_df.head()

In [ ]:
bioVillage_df = bioVillage_df.dropna(subset=['brands'])

In [ ]:
bioVillage_df['new_brands'] = bioVillage_df['brands'].apply(lambda x: clean_brand(x))

In [ ]:
bioVillage_df = bioVillage_df[bioVillage_df['new_brands'] == 'Bio Village']

In [ ]:
count = bioVillage_df.groupby('pnns_groups_1').count()[['brands']].rename(columns={'brands':'count'})
count.plot(kind='bar')
plt.title('Food category repartition for the brand Bio Village')
plt.xlabel('Food category') 
plt.show()

## Nutrition grade repartition per category

In [ ]:
category_vs_nutrition = data.select('nutrition_grade_fr', 'pnns_groups_1').na.drop()

In [ ]:
category_vs_nutrition = category_vs_nutrition.withColumn('pnns_groups_1',(F.regexp_replace('pnns_groups_1', '-', ' ')))
square_udf_int = F.udf(lambda z: z.title())
category_vs_nutrition = category_vs_nutrition.withColumn('pnns_groups_1', square_udf_int('pnns_groups_1'))

In [ ]:
category_vs_nutrition_df = category_vs_nutrition.toPandas()

In [ ]:
#category_vs_nutrition_df.to_pickle('category_vs_nutrition_df')
category_vs_nutrition_df = pd.read_pickle('category_vs_nutrition_df')

In [ ]:
category_vs_nutrition_df = pd.get_dummies(category_vs_nutrition_df.set_index('pnns_groups_1')).reset_index()
category_vs_nutrition_df = category_vs_nutrition_df.rename(columns={'nutrition_grade_fr_a':'a',
                               'nutrition_grade_fr_b':'b',
                               'nutrition_grade_fr_c':'c',
                               'nutrition_grade_fr_d':'d',
                               'nutrition_grade_fr_e':'e'})

In [ ]:
count = category_vs_nutrition_df.groupby('pnns_groups_1').sum()
count['total'] = count.apply(lambda x: x['a']+x['b']+x['c']+x['d']+x['e'], axis=1) 
count.head()

In [ ]:
ratio = count.copy()
ratio[['a','b','c','d','e']] = count[['a','b','c','d','e']].div(count['total'].values,axis=0)*100
ratio.head()

In [ ]:
ratio.reset_index().drop('total',1).sort_values(by=['a'],ascending=0)\
.plot(kind='bar', x='pnns_groups_1',figsize=[10,7], color=sns.color_palette("RdBu_r", 5))
plt.xlabel('Food Category')
plt.ylabel('proportion (in %)')
plt.title('Health profile for each food category.')
plt.show()